### Importing the libraries

In [14]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

 # map rendering library
!pip install folium
import folium

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Scraping the web and preparing the table.

In [15]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    

    if row != [] and row[1] != "Not assigned":


            if "Not assigned" in row[2]: 
                 row[2] = row[1]
            res.append(row)
           

df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])


df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")


indexNum = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNum, inplace = True)

### Importing the geographical coordinates table of each "M" group postal code of Toronto/Canada.

In [16]:
df_geo_coor = pd.read_csv("https://cocl.us/Geospatial_data")

# if you wish the look the 
# df_geo_coor.head()

### Merging the tables

In [17]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')

# removing the mutual "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)

## and.. now we know; what is the residential area's coordinates..

In [18]:
df_toronto.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
